In [ ]:
!pip install google-api-python-client
!pip install google-auth
!pip install google-auth-httplib2
!pip install google-auth-oauthlib



In [1]:
!pip install youtube_transcript_api

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pickle
import os
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
import json 

scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]

def Create_Service(client_secret_file, api_name, api_version, *scopes):
    print(client_secret_file, api_name, api_version, scopes, sep="-")
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes[0]]
    print(SCOPES)
    cred = None
    pickle_file = f"token_{API_SERVICE_NAME}_{API_VERSION}.pickle"

    if os.path.exists(pickle_file):
        with open(pickle_file, "rb") as token:
            cred = pickle.load(token)

    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            cred = flow.run_local_server()

        with open(pickle_file, "wb") as token:
            pickle.dump(cred, token)
            print("create token file ===>",pickle_file )

    try:
        service = build(API_SERVICE_NAME, API_VERSION, credentials=cred)
        print(API_SERVICE_NAME.capitalize(), "service created successfully.\n")
        return service
    except Exception as e:
        print("Unable to connect.")
        print(e)
        return None

#khởi tạo dịch vụ để sử dung Youtube API
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
api_service_name = "youtube"
api_version = "v3"
client_secrets_file = "client_secret_youtube.json"
#Tạo dịch vụ
service = Create_Service(client_secrets_file, api_service_name, api_version, scopes)

# Lấy thông tin các video liên quan theo từ khóa tìm kiếm
request = service.search().list(
    part="snippet",
    maxResults=100,
    q="chiến sự tại ukraine" # Sửa keyword để tìm kiếm tại đây
)
response = request.execute()

print(response)
#Ghi ra file để tiện theo dõi
with open("result.json", "w", encoding='utf-8') as f:
    json.dump(response, f, ensure_ascii=False, indent=4)

# dùng luôn thư viện để lấy caption cho tiện
from youtube_transcript_api import YouTubeTranscriptApi

result_all = []
response_items = response["items"]
for itemvideo in response_items:
  if(itemvideo["id"]["kind"] == 'youtube#video'):
    #Lấy videoID để lấy captions của video
    videoid = itemvideo["id"]["videoId"]
    
    #Liệt kê ds các captions trong video và lấy captions tiếng việt
    transcript_fulltxt=[]
    transcript_list = YouTubeTranscriptApi.list_transcripts(videoid)
    for transcript in transcript_list:
        #có thể sử ngôn ngữ đầu ra tại đây
        transcript_fulltxt = transcript.translate('vi').fetch()
    onlyTrans = [item['text'] for item in transcript_fulltxt]
    # Gom tất cả đoạn caption thành một chuỗi duy nhất
    dataTrans = " ".join(onlyTrans)

    # dữ liệu tạm để lưu trữ 
    tempdata = {
        "videoId": itemvideo["id"]["videoId"],
        "publishedAt": itemvideo["snippet"]["publishedAt"],
        "title": itemvideo["snippet"]["title"],
        "channel" : {
            "channelID": itemvideo["snippet"]["channelId"],
            "channelTitle": itemvideo["snippet"]["channelTitle"],
        },
        "description" : itemvideo["snippet"]["description"],
        "captionData" : dataTrans
    }
    print(tempdata)
    result_all.append(tempdata)



client_secret_youtube.json-youtube-v3-(['https://www.googleapis.com/auth/youtube.force-ssl'],)
['https://www.googleapis.com/auth/youtube.force-ssl']
Youtube service created successfully.

{'kind': 'youtube#searchListResponse', 'etag': 'juE5_M5RS0E7Q9Pmr_njbAUcEds', 'nextPageToken': 'CDIQAA', 'regionCode': 'VN', 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 50}, 'items': [{'kind': 'youtube#searchResult', 'etag': 'Drjy-5GRR6eIkovDbygVAqkQnBw', 'id': {'kind': 'youtube#video', 'videoId': 'okK8S3UpfLQ'}, 'snippet': {'publishedAt': '2024-03-06T06:22:27Z', 'channelId': 'UClYUCB6tyZl50_7OcbgUHCA', 'title': 'Cập nhật chiến sự Nga - Ukraine | Tin thế giới mới nhất hôm nay', 'description': 'Nga Ukraine mới nhất | Cực nóng: Thủ tướng nước NATO vào diện đặc biệt bị nghi ngờ âm thầm giúp Nga Phó Chủ tịch Ủy ban ...', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/okK8S3UpfLQ/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/okK8S3UpfLQ/mqdefaul

TranscriptsDisabled: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=hC2uFxuIYOc! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!

In [ ]:
import io

with io.open('result_new.json', 'w', encoding='utf-8') as f:
    json.dump(result_all, f, ensure_ascii=False, indent=4)